In [2]:
# Installation cell
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

--2020-05-27 11:42:12--  https://julialang-s3.julialang.org/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz
Resolving julialang-s3.julialang.org (julialang-s3.julialang.org)... 151.101.2.49, 151.101.66.49, 151.101.130.49, ...
Connecting to julialang-s3.julialang.org (julialang-s3.julialang.org)|151.101.2.49|:443... connected.
HTTP request sent, awaiting response... 302 gce internal redirect trigger
Location: https://storage.googleapis.com/julialang2/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz [following]
--2020-05-27 11:42:12--  https://storage.googleapis.com/julialang2/bin/linux/x64/1.3/julia-1.3.1-linux-x86_64.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c1b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95929584 (91M) [application/x-gzip]
Saving to: ‘/tmp/julia.tar.gz’

/tmp/julia.tar.gz   100%[===================

In [15]:
using Pkg
Pkg.add(["Knet","LinearAlgebra", "Random", "SparseArrays", "Statistics","ScikitLearn"])

 Resolving package versions...
  Updating `~/.julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.3/Manifest.toml`
 [no changes]


# Import Related Libraries

In [0]:
using Knet, LinearAlgebra, Random, SparseArrays, Statistics, ScikitLearn


Create network weights struct:



In [17]:
mutable struct network_weights
    distance_parameter;
    input_dim;
    stride;
    NpS;
    output_dim;
    W;
    L;
    W_structure;
    L_structure;
    previous_NpS;
    lateral_distance;
end
    

# defining struct constants for W matrix
function create_h_distances(n::network_weights)
    distances = zeros(n.output_dim^2, n.input_dim, n.input_dim)
    dict_input_2_position = Dict()
    for row_index in 1:n.input_dim
        for column_index in 1:n.input_dim
            input_index = row_index*n.input_dim + column_index
            dict_input_2_position[row_index, column_index] = input_index
        end
    end
    centers = []
    dict_output_2_position = Dict()
    for i in 1:n.output_dim
        for j in 1:n.output_dim
            stride_padding = n.stride/2
            neuron_center = [i*n.stride + stride_padding, j*n.stride + stride_padding]
            push!(centers,neuron_center)
            neuron_index = (i-1)*n.output_dim + j
            dict_output_2_position[neuron_index] = neuron_center
            for k in 1:n.input_dim
                 for l in 1:n.input_dim
                    #size issue here neuron index becomes 50 where it should be 49
                    distances[neuron_index, k,l] = norm([k+0.5,l+0.5]-neuron_center)
                 end
           end
        end
    end
    above_threshold = distances .> n.distance_parameter
    below_threshold = distances .<= n.distance_parameter
    distances[above_threshold] .= 0
    distances[below_threshold] .= 1
    distances = reshape(distances,(n.output_dim^2, n.input_dim^2))
    return distances
end

# defining structure constants for L matrix
function create_ah_distances(n::network_weights)
    centers = []
    dict_output_2_position = Dict()
    for i in 1:n.output_dim
        for j in 1:n.output_dim
            stride_padding = n.stride/2
            neuron_center = [(i-1)*n.stride + stride_padding, (j-1)*n.stride + stride_padding]
            push!(centers,neuron_center)
            neuron_index = (i-1)*n.output_dim + j
            dict_output_2_position[neuron_index] = neuron_center
        end
    end
    distances_ah = zeros(n.output_dim^2, n.output_dim^2)
    for row_index in keys(dict_output_2_position)
        center = dict_output_2_position[row_index]
        for column_index in keys(dict_output_2_position)
            other_center = dict_output_2_position[column_index]
            distances_ah[row_index, column_index] = norm(other_center - center)
        end
    end
    above_threshold = distances_ah .> n.lateral_distance #*self.anti_hebbian_binary
    below_threshold = distances_ah .<= n.lateral_distance #*self.anti_hebbian_binary
    distances_ah[above_threshold] .= 0
    distances_ah[below_threshold] .= 1
    return distances_ah
end    

# Create lateral connections
function create_L(n::network_weights)
    mat = create_ah_distances(n)
    L_mat = repeat(mat,n.NpS,n.NpS) ## !! Figure this one out
    return L_mat
end

# Create feedforward connections 
function create_W(n::network_weights)
    mat = create_h_distances(n)
    W_mat = repeat(mat,n.NpS,n.previous_NpS) ## !! Figure this one out
    return W_mat
end
    
# Create weights matrices    
function create_weights_matrix(n::network_weights)
    n.W_structure = create_W(n)
    n.L_structure = create_L(n)
    factor = sqrt(((sum(n.W_structure)/n.NpS)/(n.output_dim^2)))
    n.W = n.W_structure.*randn(size(n.W_structure))/factor
    n.L = n.L_structure.*Matrix{Float64}(I,n.NpS * n.output_dim^2,n.NpS * n.output_dim^2)
end


create_weights_matrix (generic function with 1 method)

Activation function:

In [18]:
# Activation function
function activation_function(vec)
    tanh.(vec)
end

activation_function (generic function with 1 method)

GPU check and convert to GPU function

In [19]:
#Check GPU availability
println(gpu()>=0)
if gpu() >= 0  # gpu() returns a device id >= 0 if there is a GPU, -1 otherwise 
    atype = KnetArray{Float32}  # KnetArrays are stored and operated in the GPU
end
# Converts arrays to Knet arrays to work on GPU
function convert_to_GPU(vec)
    vec=convert(atype,vec)
end
# Convert and floor to integer
function int(x)
convert(Int,floor(x))
end


true


int (generic function with 1 method)

# Network object

In [20]:
# Network object
mutable struct deep_network_GPU
    image_dim;                  # input image dimensions
    channels;                   # number of channels
    NpSs;                       # number of neurons per site
    strides;                    # stride array
    distances;                  # feedforward connections distance threshold
    lateral_distances;          # lateral connections distance threshold
    layers;                     # number of layers
    gamma;                      # feedback connections parameter
    lr;                         # learning rate
    lr_floor;                   # min learning rate
    current_lr;                 # current learning rate
    decay;                      # decay parameter of learning rate
    conversion_tickers;         # checks if neural dynamics converged
    epoch;                      # epoch counter
    structure;          
    deep_matrix_weights;        # Weights matrix
    deep_matrix_structure;      # Structure matrix
    deep_matrix_identity;       # 
    weights_adjustment_matrix;  #
    weights_update_matrix;      #
    n_images;                   #
    dict_weights;               #
    dimensions;                 #
    euler_step;                 #
    tanh_factors;               #
    mult_factors;               #
    W_gpu;                      #
end

# Create deep network
function create_deep_network(dn::deep_network_GPU)
    for i in 1:dn.layers+1
        if i==1
            dim=1
        else
            dim = int(prod(dn.strides[1:i-1]))
        end
        push!(dn.dimensions,(int((dn.image_dim/dim)^2)*hcat([dn.channels],dn.NpSs)[i]))
    end

    for i in 1:dn.layers
        if i==1
            dim=1
        else
            dim = int(prod(dn.strides[1:i-1]))
        end
        layer_input_dim = int(dn.image_dim/dim)

         dn.dict_weights[i]=network_weights(dn.distances[i],layer_input_dim,dn.strides[i],hcat([dn.channels],dn.NpSs)[i+1],
         int(layer_input_dim/dn.strides[i]),[],[],
         [], [], hcat([dn.channels],dn.NpSs)[i], dn.lateral_distances[i])
         create_weights_matrix(dn.dict_weights[i])

    end

    matrix_block = []
    structure_block = []
    matrix_identity = []
    weight_adjustment_block = []
    gradient_update_block = []
    abs_structure_block = []
    for (i, ele_row) in enumerate(dn.dimensions)
        #println("inside the loop")
        #println(i)
        #println(ele_row)
        row_block = []
        struc_block = []
        row_identity_block = []
        weights_adj_block = []
        grad_update_block = []
        abs_struc_block = []
        start_block = max(i-1, 1)

        end_block = max(length(dn.dimensions)-start_block-2, 1)
        if i == 1
            #println("pushed 1 element to row_block")
            push!(row_block,zeros(ele_row, sum(dn.dimensions)))
            push!(struc_block,zeros(ele_row, sum(dn.dimensions)))
            push!(row_identity_block, zeros(ele_row, sum(dn.dimensions)))
            push!(weights_adj_block, zeros(ele_row, sum(dn.dimensions)))
            push!(grad_update_block, zeros((ele_row, sum(dn.dimensions))))
            push!(abs_struc_block, zeros((ele_row, sum(dn.dimensions))))
        elseif i == length(dn.dimensions)
            #i=2 here
            if start_block > 1
                push!(row_block,zeros(ele_row, int(sum(dn.dimensions[1:start_block]))))
                push!(struc_block,zeros(ele_row, int(sum(dn.dimensions[1:start_block]))))
                push!(row_identity_block,zeros(ele_row, int(sum(dn.dimensions[1:start_block]))))
                push!(weights_adj_block,zeros(ele_row, int(sum(dn.dimensions[1:start_block]))))
                push!(grad_update_block,zeros(ele_row, int(sum(dn.dimensions[1:start_block]))))
                push!(abs_struc_block,zeros(ele_row, int(sum(dn.dimensions[1:start_block]))))
                #println("second elseif first if")
                #println(size(zeros(ele_row, int(sum(dn.dimensions[1:start_block])))))
            end
            #println("pushed 1 element to row_block")
            push!(row_block,dn.dict_weights[i-1].W)
            #println("pushed 1 element to row_block")
            push!(row_block,dn.dict_weights[i-1].L)
            push!(struc_block,dn.dict_weights[i-1].W_structure/dn.mult_factors[i-1])
            push!(struc_block,-dn.dict_weights[i-1].L_structure)

            push!(abs_struc_block,dn.dict_weights[i-1].W_structure)
            push!(abs_struc_block,dn.dict_weights[i-1].L_structure)

            push!(row_identity_block,zeros(size(dn.dict_weights[i-1].W_structure)))
            push!(row_identity_block,Matrix{Float64}(I,size(dn.dict_weights[i-1].L_structure,1),size(dn.dict_weights[i-1].L_structure,1))) #identity matrix

            push!(weights_adj_block,dn.dict_weights[i-1].W_structure)
            push!(weights_adj_block,dn.dict_weights[i-1].L_structure)

            push!(grad_update_block,dn.dict_weights[i-1].W_structure)
            push!(grad_update_block,dn.dict_weights[i-1].L_structure/2)

        end
        # we only push row_block to matrix_block. Check row_block
        push!(matrix_block,row_block)
        push!(structure_block,struc_block)
        push!(matrix_identity,row_identity_block)
        push!(weight_adjustment_block,weights_adj_block)
        push!(gradient_update_block,grad_update_block)
        push!(abs_structure_block,abs_struc_block)
    end


    dn.deep_matrix_weights = convert_to_GPU(vcat(matrix_block[1][1],hcat(matrix_block[2][1],matrix_block[2][2]))) 
    dn.deep_matrix_structure = convert_to_GPU(vcat(structure_block[1][1],hcat(structure_block[2][1],structure_block[2][2])))
    dn.deep_matrix_identity = convert_to_GPU(vcat(matrix_identity[1][1],hcat(matrix_identity[2][1],matrix_identity[2][2])))
    dn.weights_adjustment_matrix = convert_to_GPU(vcat(weight_adjustment_block[1][1],hcat(weight_adjustment_block[2][1],weight_adjustment_block[2][2])))
    dn.weights_update_matrix = convert_to_GPU(vcat(gradient_update_block[1][1],hcat(gradient_update_block[2][1],gradient_update_block[2][2])))
    dn.structure = convert_to_GPU(vcat(abs_structure_block[1][1],hcat(abs_structure_block[2][1],abs_structure_block[2][2])))

    println("deep_network created")
end




create_deep_network (generic function with 1 method)

# Neural Dynamics

In [21]:
# First define the neural dynamics
function neural_dynamics(dn::deep_network_GPU,img)
    conversion_ticker = 0
    x = img
    u_vec = convert_to_GPU(zeros(sum(dn.dimensions)))
    #representation vector
    r_vec = convert_to_GPU(zeros(sum(dn.dimensions)))
    
    r_vec[1:dn.channels*dn.image_dim^2] .= x
    delta = repeat([1e10],dn.layers) 

    dn.W_gpu = dn.deep_matrix_weights.*dn.deep_matrix_structure .+ dn.deep_matrix_identity
    updates = 0
    while updates < 3000
        if sum(delta .< 1e-4) == length(delta)
            conversion_ticker=1
            break
        end
        lr = max((dn.euler_step/(1+0.005*updates)), 0.05)
        delta_u = -u_vec .+ dn.W_gpu*r_vec
        u_vec[dn.channels*(dn.image_dim)^2:end] += lr*delta_u[dn.channels*dn.image_dim^2:end]
        r_vec[dn.channels*(dn.image_dim)^2:end] = activation_function(u_vec[dn.channels*dn.image_dim^2:end])
        updates += 1
        if (updates+1)%100 == 0
            # may be problems with indexing here. made changes here that may not work under different modes.
            #println(size(dn.dimensions[2:end]))
            for layer in 1:dn.layers
                start_token_large = sum(dn.dimensions[1:layer])
                end_token_large = sum(dn.dimensions[1:layer+1])
                start_token_small = int(sum(dn.dimensions[2:end])) 
                end_token_small = sum(dn.dimensions[2:end]) 
                delta_layer = norm(delta_u[start_token_small:end_token_small])/norm(u_vec[start_token_large:end_token_large])
                delta[layer] = delta_layer
            end
        end
    end
    return r_vec, conversion_ticker
end


neural_dynamics (generic function with 1 method)

# Weight Updates

In [22]:

# Update weights via stochastic gradient ascent-descent
function update_weights(dn::deep_network_GPU,r_vec)
     dn.current_lr = max(dn.lr/(1+dn.decay*dn.epoch), dn.lr_floor)
     update_matrix = r_vec*r_vec' #update for L matrix
     grad_weights = convert_to_GPU(dn.weights_update_matrix).*(update_matrix - convert_to_GPU(dn.weights_adjustment_matrix.*dn.deep_matrix_weights))
     dn.deep_matrix_weights += dn.current_lr*grad_weights
end


update_weights (generic function with 1 method)

# Training

In [23]:
# training function to call neural dynamics and gradient ascent-descent 
function training(dn::deep_network_GPU, images)
     dn.n_images = size(images,1)
         sum_ticker = 0
         for img in 1:size(images,1)
             r, conversion_ticker = neural_dynamics(dn,images[img,:])
             sum_ticker += conversion_ticker
             update_weights(dn,r)
         end
         dn.epoch+=1
         push!(dn.conversion_tickers,sum_ticker/dn.n_images)
         
         # Commented these out for speed
    
         println("")
         println("Epoch: "*string(dn.epoch))
         println("Conversion: "*string(dn.conversion_tickers[end]))
         println("Current Learning Rate: "*string(dn.current_lr)) 
         println("")
 end        

training (generic function with 1 method)

# Import Data

In [24]:
# import mnist and preprocess

function get_mnist()
    
    #dtrn,dtst = mnistdata(batchsize=1;xsize=(784,:),xtype=atype) # The setting is online so batchsize is set to 1.
    X_train, y_train, X_test, y_test = mnist() # loading the data
    X_train = convert(atype,reshape(X_train, 784, 60000)')
    X_test = convert(atype,reshape(X_test, 784, 10000)')
    
     # preprocess data
     X_train =X_train .- mean(X_train;dims=2)
     X_test =X_test.- mean(X_test;dims=2)
    
    #summary.(first(X_train))
    return X_train, X_test, y_train, y_test
end


get_mnist (generic function with 1 method)

# Main function:

In [25]:
include(Knet.dir("data","mnist.jl"))  # Load data
x_train, x_test, y_train, y_test = get_mnist()


# network parameters
NpSs=[4]
image_dim = 28
channels = 1
strides = [2]
distances = [4]
tanh_factors = 1 
layers = length(distances)
distances_lateral =  repeat([0],layers)
gamma = 0
mult_factors = 1

lr=5e-3;
lr_floor = 1e-4;
decay=0.5
conversion_tickers = []
epoch =0
structure =[]
deep_matrix_weights= [];
deep_matrix_structure = [];
deep_matrix_identity= [] ;
weights_adjustment_matrix= [];
weights_update_matrix= [];
grad_matrix =[];
n_images =[];
dict_weights = Dict()
dimensions = [];
euler_step=0.2;
tanh_factors=1;
mult_factors=1;
W_gpu=[];

println("initializing the network")

    network= deep_network_GPU(image_dim,channels,NpSs,strides,distances,distances_lateral, layers,gamma, lr,lr_floor,
                            lr, decay,conversion_tickers, epoch, structure, deep_matrix_weights,
                            deep_matrix_structure, deep_matrix_identity, weights_adjustment_matrix,
                            weights_update_matrix, n_images, dict_weights, dimensions,
                            euler_step, tanh_factors,mult_factors , W_gpu);
    
    println("network defined")

    create_deep_network(network)
    
    println("NpS: "*string(network.NpSs))
    println("Strides: "*string(network.strides))
    println("Distances: "*string(network.distances))
    println("Lateral Distances: "*string(network.lateral_distances))
    println("gamma : "*string(network.gamma))
    println("tanh_factor : "*string(network.tanh_factors))
    
    no_of_epochs=120
    for i in 1:no_of_epochs
        indices = 1:size(x_train,1)
        rand_indices = rand(indices, 1000)
        x_train_rand = x_train[rand_indices,:]
        training(network, x_train_rand)
    end
    x_train_rand=nothing

println("DONE")

initializing the network
network defined
deep_network created
NpS: [4]
Strides: [2]
Distances: [4]
Lateral Distances: [0]
gamma : 0
tanh_factor : 1

Epoch: 1
Conversion: 1.0
Current Learning Rate: 0.005


Epoch: 2
Conversion: 1.0
Current Learning Rate: 0.0033333333333333335


Epoch: 3
Conversion: 1.0
Current Learning Rate: 0.0025


Epoch: 4
Conversion: 1.0
Current Learning Rate: 0.002


Epoch: 5
Conversion: 1.0
Current Learning Rate: 0.0016666666666666668


Epoch: 6
Conversion: 1.0
Current Learning Rate: 0.0014285714285714286


Epoch: 7
Conversion: 1.0
Current Learning Rate: 0.00125


Epoch: 8
Conversion: 1.0
Current Learning Rate: 0.0011111111111111111


Epoch: 9
Conversion: 1.0
Current Learning Rate: 0.001


Epoch: 10
Conversion: 1.0
Current Learning Rate: 0.0009090909090909091


Epoch: 11
Conversion: 1.0
Current Learning Rate: 0.0008333333333333334


Epoch: 12
Conversion: 1.0
Current Learning Rate: 0.0007692307692307692


Epoch: 13
Conversion: 1.0
Current Learning Rate: 0.0007142857

# Learn Representations

In [26]:

# learn representations for tranining data
train_representations=[]
#for i in  1:int(size(x_train,1))
N_m=60000
for i in 1:N_m
    train_rep, _ = neural_dynamics(network,x_train[i,:])
     push!(train_representations, train_rep)
 end

# clear train data from memory
x_train=nothing
Knet.gc()

# this conversion is needed for ScikitLearn
for i in 1:length(train_representations)
    train_representations[i]=convert(Array{Float64,1},train_representations[i])
end
train_representations=hcat(train_representations...)'

60000×1568 Adjoint{Float64,Array{Float64,2}}:
 -0.13768    -0.13768    -0.13768    …  -0.352322  -0.338648  0.238573
 -0.155537   -0.155537   -0.155537      -0.390651  -0.374094  0.264187
 -0.0972539  -0.0972539  -0.0972539     -0.257321  -0.249663  0.175386
 -0.0857093  -0.0857093  -0.0857093     -0.228335  -0.22216   0.156062
 -0.116116   -0.116116   -0.116116      -0.303     -0.292663  0.205758
 -0.148064   -0.148064   -0.148064   …  -0.374893  -0.35955   0.253639
 -0.0882653  -0.0882653  -0.0882653     -0.234814  -0.228322  0.160386
 -0.179407   -0.179407   -0.179407      -0.438264  -0.417826  0.296261
 -0.0543918  -0.0543918  -0.0543918     -0.146701  -0.143776  0.101142
 -0.109564   -0.109564   -0.109564      -0.287385  -0.27801   0.195381
 -0.142797   -0.142797   -0.142797   …  -0.363542  -0.349049  0.246057
 -0.0712785  -0.0712785  -0.0712785     -0.191184  -0.186658  0.131179
 -0.178877   -0.178877   -0.178877      -0.437251  -0.416899  0.295575
  ⋮                            

In [27]:
# learn representations for test data
test_representations = []
for i in 1:size(x_test,1)
#for i in 1:N_m
      test_rep, _ = neural_dynamics(network, x_test[i,:])
      push!(test_representations,test_rep)
  end

# clear test data from memory
x_test=nothing
Knet.gc()

for i in 1:length(test_representations)
    test_representations[i]=convert(Array{Float64,1},test_representations[i])
end

test_representations=hcat(test_representations...)'

10000×1568 Adjoint{Float64,Array{Float64,2}}:
 -0.0923069  -0.0923069  -0.0923069  …  -0.244988  -0.237982  0.167171 
 -0.144308   -0.144308   -0.144308      -0.258814  -0.352082  0.248244 
 -0.0493748  -0.0493748  -0.0493748     -0.133329  -0.130808  0.0920509
 -0.185144   -0.185144   -0.185144      -0.449099  -0.42774   0.303614 
 -0.0962235  -0.0962235  -0.0962235     -0.254763  -0.247243  0.173683 
 -0.0693027  -0.0693027  -0.0693027  …  -0.186029  -0.181709  0.127713 
 -0.105962   -0.105962   -0.105962      -0.278685  -0.269826  0.189598 
 -0.105352   -0.105352   -0.105352      -0.277203  -0.26843   0.188612 
 -0.153731   -0.153731   -0.153731      -0.386881  -0.370617  0.261661 
 -0.156813   -0.156813   -0.156813      -0.3933    -0.376535  0.265963 
 -0.148865   -0.148865   -0.148865   …  -0.3766    -0.361127  0.25478  
 -0.13873    -0.13873    -0.13873       -0.354641  -0.340799  0.240119 
 -0.114461   -0.114461   -0.114461      -0.299081  -0.28899   0.203154 
  ⋮               

# Clasify Using Linear SVM

In [28]:
classifier = SVC(max_iter=1e6, class_weight="balanced",tol=1e-5,random_state=0,kernel="linear")

# fit linear SVM to learned train representations

fit!(classifier, train_representations[:,end-network.dimensions[end]+1:end], y_train[1:N_m])

train_score = score(classifier, train_representations[:,end-network.dimensions[end]+1:end], y_train[1:N_m])
println("train_score")
println(train_score)
println("train_error")
println(1-train_score)


train_score
0.9922166666666666
train_error
0.007783333333333364


# Scores

In [29]:
# classify the test representations on the trained linear SVM classifier
test_score=score(classifier,test_representations[:,end-network.dimensions[end]+1:end], y_test)

println("test_score")
println(test_score)

println("test_error")
println(1-test_score)

# print number of data points used to train
println("")
println("Number of data points used to train the network")
println(N_m)

test_score
0.9679
test_error
0.03210000000000002

Number of data points used to train the network
60000
